In [6]:
# !nslookup huggingface.co
# !ping -c 4 huggingface.co
import requests

try:
    response = requests.get("https://huggingface.co")
    print("Connected, status code:", response.status_code)
except requests.ConnectionError:
    print("No internet connection.")


Connected, status code: 200


In [7]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [8]:

# load the pretrained model

model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# load BERT with a classification head 
# model = TFAutoModelForSequenceClassification.from_pretrained( model_name, num_labels = 2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# load the dataset (IMDb Dataset) 
from datasets import load_dataset 
dataset = load_dataset("imdb")


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
# tokenize the text 
def tokenize_function(examples): 
    return tokenizer(examples["text"], padding = "max_length", truncation = True) 

tokenized_datasets = dataset.map(tokenize_function, batched = True) 


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
# prepare the data for training 
import tensorflow as tf 

# convert dataset to TensorFlow format 
tokenized_datasets.set_format(type = "tensorflow", columns = ["input_ids", "attention_mask", "label"])

In [12]:
# create TensorFlow datasets 
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns = [ "input_ids", "attention_mask"],
    label_cols = ["label"],
    batch_size = 8, 
    shuffle = True
)

test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns = ["input_ids", "attention_mask"],
    label_cols = ["label"],
    batch_size = 8
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:405: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [13]:
# compile the model 
"""
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
              metrics = ["accuracy"]) 
"""

'\nmodel.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5),\n              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), \n              metrics = ["accuracy"]) \n'

In [14]:

# Enable multi-GPU training
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])

# Train the model
model.fit(train_dataset, epochs=3, validation_data=test_dataset)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
3125/3125 [==============================] - 981s 270ms/step - loss: 0.2729 - accuracy: 0.8909 - val_loss: 0.2022 - val_accuracy: 0.9266
Epoch 2/3
3125/3125 [==============================] - 796s 255ms/step - loss: 0.1553 - accuracy: 0.9439 - val_loss: 0.2188 - val_accuracy: 0.9175
Epoch 3/3
3125/3125 [==============================] - 796s 255ms/step - loss: 0.1030 - accuracy: 0.9640 - val_loss: 0.3498 - val_accuracy: 0.8776


In [ ]:
# train the model 
# model.fit(train_dataset, epochs = 3, validation_data = test_dataset) 


Epoch 1/3
3125/3125 [==============================] - 1806s 565ms/step - loss: 0.2826 - accuracy: 0.8808 - val_loss: 0.2175 - val_accuracy: 0.9154
Epoch 2/3
3125/3125 [==============================] - ETA: 0s - loss: 0.1702 - accuracy: 0.9389

In [15]:
# evaluate the model 
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

3125/3125 [==============================] - 104s 33ms/step - loss: 0.3498 - accuracy: 0.8776
Test Loss: 0.3498
Test Accuracy: 0.8776


In [16]:
# save fine-tuned model 
model.save_pretrained("bert-imdb-finetuned") 


In [17]:
# reload the model if is needed 
from transformers import TFAutoModelForSequenceClassification
# model = TFAutoModelForSequenceClassification.from_pretrained("bert-imdb-finetuned") 
